In [7]:
import spacy
import lemminflect
import ipywidgets as wd
from IPython.display import display
# import numpy as np
%matplotlib widget

In [63]:
nlp = spacy.load('en_core_web_md')

In [5]:
def stripPunct(doc_dep):
    return [w for w in doc_dep if w.tag_ != '.']

def getInfo(text):
    for w in nlp(text):
        print(w.text, w.lemma_, w.pos_, w.tag_)

In [59]:
def getSubject(doc):
    for token in doc:
        if ("subj" in token.dep_):
            subtree = list(token.subtree)
            start = subtree[0].i
            end = subtree[-1].i + 1
            return start, end

In [105]:
doc = nlp('Does the distant object exist?')
for token in doc:
    print(token.dep_)

aux
det
amod
nsubj
ROOT
punct


In [75]:
def statementify(question, negate=False):
    doc_dep = stripPunct(nlp(question))
    verbs = [(i, w) for i, w in enumerate(doc_dep) if w.tag_ in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'MD']]
    assert verbs[0][0] == 0
    if len(verbs) == 2 and verbs[0][1].lemma_ == 'do' and negate == False:
        verb_inflected = verbs[1][1]._.inflect(verbs[0][1].tag_)
        text_new = [w.text for w in doc_dep[1:verbs[1][0]]] + [verb_inflected] + [w.text for w in doc_dep[verbs[1][0]+1:]]
    else:
        start, end = getSubject(doc_dep)
        text_new = [w.text for w in doc_dep[1:end]] + [verbs[0][1].text.lower()]
        text_new += ['not'] if negate else []
        text_new += [w.text for w in doc_dep[end:]]
    return ' '.join(text_new)

In [113]:
def bayes(prior, true_pos_rate, false_pos_rate, pos):
    if pos:
        return prior * true_pos_rate / (prior * true_pos_rate + (1 - prior) * false_pos_rate)
    else:
        return prior * (1 - true_pos_rate) / (prior * (1 - true_pos_rate) + (1 - prior) * (1 - false_pos_rate))

In [79]:
statementify('Did the car make a noise?', negate=False)

'the car made a noise'

In [122]:
def q1Submit(sender=None):
    q1_textbox.disabled = True
    q1_submit.disabled = True
    q2_text = statementify(q1_textbox.value)
    q2_label.value = 'Without performing any further tests, what is the prior probability that ' + q2_text + '?'
    display(wd.VBox([q2_label, wd.HBox([q2_textbox, q2_submit])]))

def q2Submit(sender=None):
    q2_textbox.disabled = True
    q2_submit.disabled = True
    display(wd.VBox([q3_label, wd.HBox([q3_textbox, q3_submit])]))
    
def q3Submit(sender=None):
    q3_textbox.disabled = True
    q3_submit.disabled = True
    q4_text = statementify(q1_textbox.value)
    q4_label.value = 'If ' + q4_text + ', how likely would the ' + q3_textbox.value + ' correctly turn up positive?'
    display(wd.VBox([q4_label, wd.HBox([q4_textbox, q4_submit])]))

def q4Submit(sender=None):
    q4_textbox.disabled = True
    q4_submit.disabled = True
    q5_text = statementify(q1_textbox.value, negate=True)
    q5_label.value = 'If ' + q5_text + ', how likely would the ' + q3_textbox.value + ' correctly turn up negative?'
    display(wd.VBox([q5_label, wd.HBox([q5_textbox, q5_submit])]))

def q5Submit(sender=None):
    q5_textbox.disabled = True
    q5_submit.disabled = True
    q6_label.value = 'What is the actual result of the ' + q3_textbox.value + '?'
    display(wd.VBox([q6_label, wd.HBox([q6_textbox, q6_submit])]))

def q6Submit(sender=None):
    q6_textbox.disabled = True
    q6_submit.disabled = True
    q7_text = statementify(q1_textbox.value)
    bayes_val = bayes(q2_textbox.value, q4_textbox.value, 1-q5_textbox.value, pos=q6_textbox.value == 'Positive')
    q7_label.value = 'Knowing the test result, the probability that ' + q7_text + ' is ' + '{:.3f}'.format(bayes_val) + '.'
    display(q7_label)

q1_label = wd.Label(value='What is the question you are trying to answer?')
q1_textbox = wd.Text()
q1_submit = wd.Button(description='Next')
q1_submit.on_click(q1Submit)

q2_label = wd.Label(value='')
q2_textbox = wd.BoundedFloatText(value=0.5, min=0, max=1, step=0.001)
q2_submit = wd.Button(description='Next')
q2_submit.on_click(q2Submit)

q3_label = wd.Label(value='What is the test you are performing?')
q3_textbox = wd.Text()
q3_submit = wd.Button(description='Next')
q3_submit.on_click(q3Submit)

q4_label = wd.Label(value='')
q4_textbox = wd.BoundedFloatText(value=0.5, min=0, max=1, step=0.001)
q4_submit = wd.Button(description='Next')
q4_submit.on_click(q4Submit)

q5_label = wd.Label(value='')
q5_textbox = wd.BoundedFloatText(value=0.5, min=0, max=1, step=0.001)
q5_submit = wd.Button(description='Next')
q5_submit.on_click(q5Submit)

q6_label = wd.Label(value='')
q6_textbox = wd.RadioButtons(options=['Positive', 'Negative'])
q6_submit = wd.Button(description='Next')
q6_submit.on_click(q6Submit)

q7_label = wd.Label(value='')

In [124]:
for widget in [q2_textbox, q2_submit, q3_textbox, q3_submit, q4_textbox, q4_submit, q5_textbox, q5_submit, q6_textbox, q6_submit]:
    widget.disabled = False
q1_label = wd.Label(value='What is the question you are trying to answer?')
q1_textbox = wd.Text()
q1_submit = wd.Button(description='Next')
q1_submit.on_click(q1Submit)
display(wd.VBox([q1_label, wd.HBox([q1_textbox, q1_submit])]))

Label(value='Knowing the test result, the probability that the house is burning down is 0.001.')

In [119]:
q6_textbox.value

'Positive'

In [27]:
question = 'Does the patient suffer from tuberculosis?'

''''''

'they have gone mad'

In [28]:
statementify('Do I have Covid?')

'I have Covid'

In [ ]:
'''
What's the question you are trying to answer?
E.g. Do I have Covid? Is XXX the murderer?

Without performing any further tests, what is the prior probability that I have Covid/XXX is the murderer?
E.g. 50% if its a binary choice. 1/9 if there are 9 suspects.

What is the test I'm performing?
E.g. PCR Covid test. DNA matching.

If I have Covid/XXX is the murderer, how likely would the PCR Covid test/DNA matching correctly turn up positive?
(alt) ..., falsely turn up negative?

If I do not have Covid/XXX is not the murderer, how likely would the PCR Covid test/DNA matching correctly turn up negative?
(alt) ..., falsely turn up positive?

You have now performed the PCR Covid test/DNA matching. What was the result?

Conclusion: The probability that you actually have Covid/XXX actually is the murderer is (...).

Print the steps.
'''